In [3]:
import csv
import random
import itertools
import random
import numpy as np
from nltk.tokenize import word_tokenize

with open('train.tsv') as f:
    train_user=f.readlines()
    
with open('test.tsv') as f:
    test_user=f.readlines()

In [5]:
with open('docs.tsv') as f:
    newsdocs=f.readlines()

In [4]:
train_user[0].split('\t') 
#session id, user id, user type, clicked news id, dwell time, click time, history position, history size, clicked news, unclicked news, session start, session end, clicked news dwell time, clicked news position, unclicked news position, clicked news size, unclicked news size

['e778dccb-8ce2-4c9b-8a02-f07b8dd5fa7a',
 '11F54CBBA92063A33013404FAD206049',
 'M',
 'AAJ4j0s AAJfTQO AAJfSs3 AAJanm8',
 '198 154 1185 147',
 '10/21/2019 2:12:44 PM#TAB#10/24/2019 12:39:08 AM#TAB#10/24/2019 12:42:03 AM#TAB#10/25/2019 5:13:24 AM',
 '1 7 29 60',
 'RiverCardPortrait#TAB#RiverCardPhoto#TAB#RiverCardPhoto#TAB#RiverCard1uDoubleWide',
 'AAJmObr',
 'BBVcWey BBVcWey BBUPTEK AAIsqfs AAIsqfs AAEyYIZ AAEyYIZ AAHWHJu AAHWHJu AAHWLNQ AAHWLNQ AAIWw0o AAIWw0o AAJeYUF AAJeYUF AAJg2Cq AAJg2Cq AAJm4Nx AAJm4Nx AAJmUb5 AAJmUb5 AAJmVHl AAJmVHl AAJmWCa AAJmWCa AAJnP9M AAJnP9M AAJnQr4 AAJnQr4 AAJnTKk AAJnTKk AAJndUx AAJndUx AAJoEAq AAJoH3n AAJoH3n AAJoI8I AAJoI8I AAJorQN AAJorQN AAJp4iG AAJp4iG AAJpDGB AAJpDGB AAJpEvP AAJpEvP AAJpFT4 AAJpFT4 AAJpXzC AAJpXzC AAJpcmG AAJpcmG AAJpdjE AAJpdjE AAJpqT8 AAJpqT8 AAJprX2 AAJprX2 AAJptS7 AAJptS7 AAJpwLv AAJpwLv AAJpy1c AAJpy1c AAJpyYS AAJpyYS AAJhvE5 AAJhvE5 AAJjFl3 AAJjFl3 AAJjFl3 AAGdFwT AAGdFwT AAIMGe6 AAG8tJh',
 '10/27/2019 3:38:49 AM',
 '10/27/201

In [7]:
newsdocs[0].split('\t') 
#newsid, category, subcategory, title, body

['AA8lMqC',
 'news',
 'newsscienceandtechnology',
 'Spectacular photos from space',
 'Curiosity\'s selfie and other amazing photos from space this week. Curiosity\'s selfie and other amazing photos from space this week. Cosmic Pretzel The Atacama Large Millimeter/submillimeter Array (ALMA) captured this unprecedented image of two circumstellar disks, in which baby stars are growing, feeding with material from their surrounding birth disk. The complex network of dust structures distributed in spiral shapes remind of the loops of a pretzel. These observations shed new light on the earliest phases of the lives of stars and help astronomers determine the conditions in which binary stars are born. Iberian Nights The Iberian Peninsula at night, showing Spain and Portugal. Madrid is the bright spot just above the center. Dark World This artist\'s illustration depicts the exoplanet LHS 3844b, which is 1.3 times the mass of Earth and orbits an M dwarf star. The planet\'s surface may be covered 

In [ ]:
news={}
for doc in newsdocs:
    line=doc.split('\t')
    #category, subcategory, title 
    #only title is used later
    news[line[0]]=[line[1],line[2],word_tokenize(line[3].lower())]

news_index={'NULL':0}
for docid in news:
    news_index[docid]=len(news_index)

In [ ]:
position_dict={'padding':-1}
for i in train_user+test_user: 
    tp=i.replace('\n','').split('\t')
    for j in tp[6].split():
        if j not in position_dict:
            position_dict[j]=len(position_dict)

In [ ]:
#some sessions are too short to sample 4 unclicked news
def new_sample(input_list,ratio):
    if ratio >len(input_list):
        return random.sample(input_list*(ratio//len(input_list)+1),ratio)
    else:
        return random.sample(input_list,ratio)

In [ ]:
npratio=4
MAX_HIS_LENGTH=50
train_candidate=[]    
train_label=[]
train_candidate_pos=[] 
train_candidate_pos_rand=[] 
train_his_pos=[]
train_his=[]

for i in train_user:
  
    line=i.replace('\n','').split('\t')

    clickids=[news_index[x] for x in line[3].split()][-MAX_HIS_LENGTH:]
    position=[int(x) for x in line[6].split()][-MAX_HIS_LENGTH:]
    click_doc=[news_index[x] for x in line[8].split()]
    unclick_doc=[news_index[x] for x in line[9].split()]
    click_doc_pos=[int(x) for x in line[13].split()]
    unclick_doc_pos=[int(x) for x in line[14].split()]
    click_doc=list(zip(click_doc,click_doc_pos))
    unclick_doc=list(zip(unclick_doc,unclick_doc_pos))
    
    for t in range(len(click_doc)):
        positive=click_doc[t]
        negatives=new_sample(unclick_doc,npratio)
        sample=negatives+[positive]
        label=[0]*npratio+[1]
        label_index=list(range(npratio+1))
        random.shuffle(label_index)
        sample_shuffle=[]
        label_shuffle=[]
        for j in label_index:
            sample_shuffle.append(sample[j])
            label_shuffle.append(label[j])
            
        train_candidate.append([x[0] for x in sample_shuffle])
        train_candidate_pos.append([x[1] for x in sample_shuffle])
        train_candidate_pos_rand.append([random.randint(0,len(position_dict)-1) for x in sample_shuffle])
        train_label.append(label_shuffle)
        train_his.append(clickids+[0]*(MAX_HIS_LENGTH-len(clickids)))
        train_his_pos.append(position+[0]*(MAX_HIS_LENGTH-len(position)))


test_candidate=[]    
test_label=[]
test_his=[]
test_his_pos=[]
test_candidate_pos=[]   
test_index=[]


for i in test_user:
  
    line=i.replace('\n','').split('\t')

    clickids=[news_index[x] for x in line[3].split()][-MAX_HIS_LENGTH:]
    position=[int(x) for x in line[6].split()][-MAX_HIS_LENGTH:]
    
    click_doc=[news_index[x] for x in line[8].split()]
    unclick_doc=[news_index[x] for x in line[9].split()]
    click_doc_pos=[int(x) for x in line[13].split()]
    unclick_doc_pos=[int(x) for x in line[14].split()]
    click_doc=list(zip(click_doc,click_doc_pos))
    unclick_doc=list(zip(unclick_doc,unclick_doc_pos))
    current_index=[]
    current_index.append(len(test_candidate))
    
    for t in range(len(click_doc)):
        sample=click_doc[t]
        test_candidate.append(sample[0])
        test_candidate_pos.append(sample[1])
        test_label.append(1)
        test_his.append(clickids+[0]*(MAX_HIS_LENGTH-len(clickids)))
        test_his_pos.append(position+[0]*(MAX_HIS_LENGTH-len(position)))
        
    for t in range(len(unclick_doc)):
        sample=unclick_doc[t]
        test_candidate.append(sample[0])
        test_candidate_pos.append(sample[1])
        test_label.append(0)
        test_his.append(clickids+[0]*(MAX_HIS_LENGTH-len(clickids)))
        test_his_pos.append(position+[0]*(MAX_HIS_LENGTH-len(position)))
        
    current_index.append(len(test_candidate))
    test_index.append(current_index)

In [ ]:
word_dict={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]:
        if j in word_dict:
            word_dict[j][1]+=1
        else:
            word_dict[j]=[len(word_dict),1]


emb_dict={}
with open('glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        line=f.readline()
        if len(line)==0:
            break
        line = line.split()
        word=line[0].decode()
        linenb+=1
        if len(word) != 0:
            vec=[float(x) for x in line[1:]]
            if word in word_dict:
                emb_dict[word]=vec



from numpy.linalg import cholesky

emb_mat=[0]*len(word_dict)

in_vocab_emb=[]
for i in emb_dict:
    emb_mat[word_dict[i][0]]=np.array(emb_dict[i],dtype='float32')
    in_vocab_emb.append(emb_mat[word_dict[i][0]])
in_vocab_emb=np.array(in_vocab_emb,dtype='float32')

mu=np.mean(in_vocab_emb, axis=0)
Sigma=np.cov(in_vocab_emb.T)


for i in range(len(emb_mat)):
    if type(emb_mat[i])==int:
        norm=np.random.multivariate_normal(mu, Sigma, 1)
        emb_mat[i]=np.reshape(norm, 300)
        
emb_mat[0]=np.zeros(300,dtype='float32')
emb_mat=np.array(emb_mat,dtype='float32')

In [ ]:
news_words=[[0]*MAX_SENT_LENGTH]

for i in news:
    newsids=[]
    for word in news[i][2]:
        if word in word_dict:
            newsids.append(word_dict[word][0])
    newsids=newsids[:MAX_SENT_LENGTH]
    news_words.append(newsids+[0]*(MAX_SENT_LENGTH-len(newsids)))

import numpy as np
news_words=np.array(news_words,dtype='int32') 

train_candidate=np.array(train_candidate,dtype='int32')
train_candidate_pos=np.array(train_candidate_pos,dtype='int32')
train_candidate_pos_rand=np.array(train_candidate_pos_rand,dtype='float32')
train_label=np.array(train_label,dtype='int32')
train_his=np.array(train_his,dtype='int32')
train_his_pos=np.array(train_his_pos,dtype='int32')


test_candidate=np.array(test_candidate,dtype='int32')
test_candidate_pos=np.array(test_candidate_pos,dtype='int32')
test_label=np.array(test_label,dtype='int32')
test_his=np.array(test_his,dtype='int32')
test_his_pos=np.array(test_his_pos,dtype='int32')

In [ ]:
from sklearn.metrics import roc_auc_score

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [ ]:
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *
from keras.layers import *


class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x

        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))

        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)

        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
def generate_batch_data_random(batch_size):
    idx = np.arange(len(train_label))
    np.random.shuffle(idx)
    batches = [idx[range(batch_size*i, min(len(train_label), batch_size*(i+1)))] for i in range(len(train_label)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_words[train_candidate[i]]
            item_pos=np.array(np.sqrt(train_candidate_pos[i]-1),dtype='int32')
            item_pos_rand=np.array(np.sqrt(train_candidate_pos_rand[i]-1),dtype='int32')
            user_his=news_words[train_his[i]]
            user_his_pos=np.array(np.sqrt(train_his_pos[i]-1),dtype='int32')
            y=train_label[i]
            
            yield ([item,item_pos,item_pos_rand,user_his,user_his_pos], [y,y])

def generate_batch_data(batch_size):

    idx = np.arange(len(test_label))
    batches = [idx[range(batch_size*i, min(len(test_label), batch_size*(i+1)))] for i in range(len(test_label)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_words[test_candidate[i]]
            item_pos=np.zeros_like(np.expand_dims(np.array(test_candidate_pos[i],dtype='int32'),axis=1))
            user_his=news_words[test_his[i]]
            user_his_pos=np.array(np.sqrt(test_his_pos[i]-1),dtype='int32')
            y=test_label[i]
            
            yield ([item,item_pos,user_his,user_his_pos], [y])

In [1]:
keras.backend.clear_session()

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_dict), 300, weights=[emb_mat],trainable=True)
embedded_sequences = embedding_layer(sentence_input)
embedded_sequences=Dropout(0.2)(embedded_sequences)

self_att = Attention(20,20)([embedded_sequences]*3) 
self_att=Dropout(0.2)(self_att)

attention = Dense(200,activation='tanh')(self_att)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
news_emb=Activation('relu')(keras.layers.Dot((1, 1))([self_att, attention_weight]))

title_encoder = Model([sentence_input], news_emb)

his_input =  Input((MAX_HIS_LENGTH, MAX_SENT_LENGTH,), dtype='int32') 
title_his=TimeDistributed(title_encoder)(his_input)


pos_input =  Input((MAX_HIS_LENGTH,), dtype='int32') 
pos_embedding_layer = Embedding(len(position_dict), 400,trainable=True)
pos_emb = posembedding_layer(pos_input)
all_emb = add([title_his,pos_emb])
user_self_att = Attention(20,20)([all_emb]*3) 


candidates =Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32') 
cand_emb=TimeDistributed(title_encoder)(candidates)


candidates_pos =Input((1+npratio,), dtype='int32') 
cand_pos=pos_embedding_layer(candidates_pos)

candidates_pos_rand =Input((1+npratio,), dtype='int32') 
cand_pos_rand=pos_embedding_layer(candidates_pos_rand)


cand_att = keras.layers.Dot((2, 2))([add([cand_emb, cand_pos]),all_emb])
cand_att = Activation('softmax')(cand_att)
user_emb=keras.layers.Dot((2, 1))([cand_att,all_emb])
user_emb=multiply([user_emb, add([cand_emb, cand_pos])])

predictor=Dense(1)
logits = Activation('softmax')(TimeDistributed(predictor)(user_emb)) 

cand_att_rand = keras.layers.Dot((2, 2))([add([cand_emb, cand_pos_rand]),all_emb])
cand_att_rand = Activation('softmax')(cand_att_rand)
user_emb_rand=keras.layers.Dot((2, 1))([cand_att_rand,all_emb])
user_emb_rand=multiply([user_emb_rand, add([cand_emb, cand_pos_rand])])
logits_rand = Activation('softmax')(TimeDistributed(predictor)(user_emb_rand))  

model = Model([candidates,candidates_pos,candidates_pos_rand,his_input,pos_input], [logits,logits_rand])

model.compile(loss=['categorical_crossentropy','categorical_crossentropy'], optimizer=Adam(lr=0.001),loss_weights=[1.,1.], metrics=['acc'])

gan_input_1=Input((1+npratio,400), dtype='float32') 
gan_input_2=Input((1+npratio,400), dtype='float32') 
gan_input=Lambda(lambda x: K.concatenate([K.expand_dims(x[0],axis=2),K.expand_dims(x[1],axis=2)],axis=2))([gan_input_1,gan_input_2])
softlogits=Activation('softmax')(Reshape((1+npratio,2))(TimeDistributed(Dense(1))(gan_input)))
biasgan = Model([gan_input_1,gan_input_2], [softlogits])

gan_output = biasgan(model([candidates,candidatespos,candidatespos_rand,his_input,pos_input]))

gan = Model([candidates,candidatespos,candidatespos_rand,his_input,pos_input], [logits,logits_rand,gan_output])

biasgan.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.0002), metrics=['acc'])#larger lr for better convergence

gan.compile(optimizer=Adam(lr=0.0001), metrics=['acc'],loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],loss_weights=[1.,1.,1.])


candidate_one = keras.Input((MAX_SENT_LENGTH,))

candidate_one_vec = title_encoder([candidate_one])
candidate_one_pos =Input((1,), dtype='int32') 
candidate_one_pos_vec=Flatten()(posembedding_layer(candidate_one_pos))

cand_att_one = keras.layers.Dot((1, 2))([add([candidate_one_vec, candidate_one_pos_vec]),all_emb])
cand_att_one = Activation('softmax')(cand_att_one)
user_emb_one=keras.layers.Dot((1, 1))([cand_att_one,all_emb])
user_emb=multiply([user_emb_one, add([candidate_one_vec, candidate_one_pos_vec])])
score = predictor(user_emb) 
model_test = keras.Model([candidate_one,candidate_onepos,his_input,pos_input], score)

for epoch in range(3):

    train_gen=generate_batch_data_random(30)
    iterations=len(train_label)//30
    step=0
    historicala=[]
    historicald=[]
    for step,data in enumerate(train_gen):
        uvec=model.predict(data[0])
        biasgan.trainable = True
        d_loss =biasgan.train_on_batch(uvec,np.array([[[1.,0.]]*5]*30))
        biasgan.trainable = False
        a_loss =gan.train_on_batch(data[0],[data[1][0],data[1][1],np.array([[[0.,1.]]*5]*30)])
        historicald.append(d_loss)
        historicala.append(a_loss)
        historicald=historicald[-100:]
        historicala=historicala[-100:]
        if step%100==0:
            print(np.mean(historicald,axis=0))
            print(np.mean(historicala,axis=0))
        if step==iterations:
            break

    valgen=generate_batch_data(30)
    pred = model_test.predict_generator(valgen, steps=len(test_candidate)//30,verbose=1)
    all_auc=[]
    all_mrr=[]
    all_ndcg5=[]
    all_ndcg10=[]
    for m in test_index:
        if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(pred):
            all_auc.append(roc_auc_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0]))
            all_mrr.append(mrr_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0]))
            all_ndcg5.append(ndcg_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0],k=5))
            all_ndcg10.append(ndcg_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0],k=10))
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg5),np.mean(all_ndcg10))